# Assesment - Universiteiten wereldwijd

## 1. Introduction

define the context, the research question and the practical relevance

UITTYPEN
Twee datasets van rankings van universiteiten wereldwijd. Ze scoren de universiteiten op verschillende onderdelen. 



Test uitvoeren met ene set en andere set

Mijn onderzoeksvraag ziet er als volgt uit: 'Kan je voorspellen hoe goed een universiteit zal scoren in de toekomst?'. Ik wil hierbij kijken naar verschillende aspecten zoals de ratio man / vrouw en het aantal studenten per docent. Als dit mogelijk is, kunnen universiteiten aanpassingen doorvoeren en zo beter scoren in de ranking. 

Top 5
6 - 20
21 - 50
51+

## 2. Dataset

explain how you acquired the data, and show your data cleaning steps

UITTYPEN
Eentje opgevraagd via website, andere gedownload.

In [1]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_csv("universities_ranking.csv") # Dataset van THE
df2 = pd.read_csv("2021_QS_World_University_Rankings_Results_public_version.csv") # Dataset van QS

In [3]:
df1.head() # Bekijken van een klein deel van de dataset

,ranking,title,location,number students,students staff ratio,perc intl students,gender ratio,overall score,teaching score,research score,citations score,industry income score,intl outlook score
0,1,University of Oxford,United Kingdom,"20,774",11.1,41%,46 : 54,95.6,91.3,99.6,98.0,68.7,96.4
1,2,Stanford University,United States,"16,223",7.4,23%,44 : 56,94.9,92.2,96.7,99.9,90.1,79.5
2,3,Harvard University,United States,"21,261",9.3,25%,49 : 51,94.8,94.4,98.8,99.4,46.8,77.7
3,4,California Institute of Technology,United States,"2,238",6.3,33%,36 : 64,94.5,92.5,96.9,97.0,92.7,83.6
4,5,Massachusetts Institute of Technology,United States,"11,276",8.4,34%,39 : 61,94.4,90.7,94.4,99.7,90.4,90.0


In [4]:
df2.head() # Bekijken van een klein deel van de dataset

,ranking,institution name,code,country,size,focus,res.,age,status,academic reputation,employer reputation,faculty student,citations per faculty,international faculty,international students,overall
0,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,B,100.0,100.0,100.0,100.0,100.0,91.4,100
1,2,University of Oxford,UK,United Kingdom,L,FC,VH,5,A,100.0,100.0,100.0,96.0,99.5,98.5,99.5
2,3=,Stanford University,US,United States,L,FC,VH,5,B,100.0,100.0,100.0,99.9,99.8,67.0,98.7
3,3=,University of Cambridge,UK,United Kingdom,L,FC,VH,5,A,100.0,100.0,100.0,92.1,100.0,97.7,98.7
4,5,Harvard University,US,United States,L,FC,VH,5,B,100.0,100.0,99.1,100.0,84.2,70.1,98


Het valt nu al op dat de datasets een andere top 5 hebben: bij de eerste staat University of Oxford op nummer één en bij de tweede staat Massachusetts Institute of Technology op nummer één. Ook verschillen de noteringen van de universiteiten. Daar moet ik later naar kijken tijdens het mergen. Eerst bekijk ik of er veel NA/s zijn.

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ranking                1526 non-null   int64  
 1   title                  1526 non-null   object 
 2   location               1526 non-null   object 
 3   number students        1526 non-null   object 
 4   students staff ratio   1526 non-null   float64
 5   perc intl students     1526 non-null   object 
 6   gender ratio           1453 non-null   object 
 7   overall score          1526 non-null   object 
 8   teaching score         1526 non-null   float64
 9   research score         1526 non-null   float64
 10  citations score        1526 non-null   float64
 11  industry income score  1526 non-null   float64
 12  intl outlook score     1526 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 155.1+ KB


De eerste dataset worden verschillende punten gebruikt om de universiteit te beoordelen. De perc intl students zijn het percentage aantal internationale studenten binnen de studie. De gender ratio zijn de verdeling van mannen en vrouwen: het linkergetal is het aantal vrouwen. De teachingscore is het cijfer voor de lessen die gegeven worden (22.000 ondervraagden). De research score is het cijfer voor de onderzoeken en de reputatie die de universiteit daarbij heeft. !!! AFMAKEN

Bij deze dataset is alleen de gender-ratio niet volledig. Het scheelt 73 scholen. Dit is wel een onderdeel wat ik graag meeneem in het onderzoek, omdat dit invloed kan hebben op hoe goed een school scoort - misschien scoort een school met meer vrouwen wel hoger dan wanneer er meer mannen zijn. 

In [6]:
df1['gender ratio'].head(20) #Kijken naar de top 20 en NaN's

0     46 : 54
1     44 : 56
2     49 : 51
3     36 : 64
4     39 : 61
5     47 : 53
6     51 : 49
7     50 : 50
8     46 : 54
9     46 : 54
10    39 : 61
11    52 : 48
12    52 : 48
13    32 : 68
14    55 : 45
15    57 : 43
16        NaN
17    59 : 41
18    50 : 50
19    49 : 51
Name: gender ratio, dtype: object

Er zitten geen NAN's in de top 20, waardoor ik er voor kies om deze weg te gooien.

In [7]:
df1 = df1.dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1453 entries, 0 to 1525
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ranking                1453 non-null   int64  
 1   title                  1453 non-null   object 
 2   location               1453 non-null   object 
 3   number students        1453 non-null   object 
 4   students staff ratio   1453 non-null   float64
 5   perc intl students     1453 non-null   object 
 6   gender ratio           1453 non-null   object 
 7   overall score          1453 non-null   object 
 8   teaching score         1453 non-null   float64
 9   research score         1453 non-null   float64
 10  citations score        1453 non-null   float64
 11  industry income score  1453 non-null   float64
 12  intl outlook score     1453 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 158.9+ KB


We houden in totaal 1453 scholen over. Nu ga ik kijken naar de tweede dataset en of er een verschil in namen en aantallen zit. 

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ranking                 1300 non-null   object 
 1   institution name        1300 non-null   object 
 2   code                    1300 non-null   object 
 3   country                 1300 non-null   object 
 4   size                    1300 non-null   object 
 5   focus                   1300 non-null   object 
 6   res.                    1300 non-null   object 
 7   age                     1300 non-null   int64  
 8   status                  1300 non-null   object 
 9   academic reputation     1300 non-null   float64
 10  employer reputation     1300 non-null   float64
 11  faculty student         1299 non-null   float64
 12  citations per faculty   1300 non-null   float64
 13  international faculty   1228 non-null   float64
 14  international students  1275 non-null   

De tweede dataset kijkt naar andere punten dan de eerste. De academic reputation is het cijfer voor de reputatie van buitenaf (130.000 ondervraagden). De employer reputation is het cijfer hoe goed de universiteiten hun studenten voorbereiden op het werkveld (75.000 ondervraagden). De faculty student ratio gaat over het aantal docenten per aantal studenten op de universiteit. De citations per faculty zijn de beoordeling van wetenschappelijke onderzoeken door een universiteit. De international faculty geeft de reputatie aan van het buitenland naar de universiteit en de international students geeft aan hoe de internationale studenten de school beoordelen. 

De international faculty heeft de minste rijen data maar is wel een belangrijk onderdeel: als blijkt dat internationale studenten een positieve invloed hebben op de score van een school, levert dit meer publiciteit op. Dan zou een school bijvoorbeeld meer werk kunnen stoppen in het werven van internationale studenten. Ik besluit die niet weg te gooien. Wel kijk ik naar de eerste 20 rijen of daar iets veranderd.

In [9]:
df2['international faculty'].head(20) #Kijken naar het eerste deel van de groepen

0     100.0
1      99.5
2      99.8
3     100.0
4      84.2
5      99.4
6     100.0
7     100.0
8      99.5
9      71.9
10    100.0
11    100.0
12     94.1
13    100.0
14     87.9
15     99.3
16     15.3
17     45.6
18     52.7
19     28.1
Name: international faculty, dtype: float64

Ook hier zijn er geen NaN's in de top 20 en besluit ik de overige NaN's weg te gooien. 

In [10]:
df2 = df2.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1225 entries, 0 to 1299
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ranking                 1225 non-null   object 
 1   institution name        1225 non-null   object 
 2   code                    1225 non-null   object 
 3   country                 1225 non-null   object 
 4   size                    1225 non-null   object 
 5   focus                   1225 non-null   object 
 6   res.                    1225 non-null   object 
 7   age                     1225 non-null   int64  
 8   status                  1225 non-null   object 
 9   academic reputation     1225 non-null   float64
 10  employer reputation     1225 non-null   float64
 11  faculty student         1225 non-null   float64
 12  citations per faculty   1225 non-null   float64
 13  international faculty   1225 non-null   float64
 14  international students  1225 non-null   

Er blijven 1225 rijen over - minder dan de andere dataframe maar niet erg. Nu ga ik kijken naar de namen en welke dataframes ik wil meenemen in mijn uiteindelijke dataframe. Ten eerste verander ik de namen van de scholen in 1 titel om ze te kunnen mergen. Ook drop ik een aantal rijen die voor mijn onderzoek niet belangrijk zijn.

In [11]:
df1 = df1.rename(columns={'location':'country'})
df2 = df2.rename(columns={'institution name':'title'})

UITLEGGEN WAAROM DEZE RIJEN

In [12]:
df1 = df1.drop(columns=['ranking'])
df2 = df2.drop(columns=['ranking', 'status', 'country', 'code', 'size', 'focus', 'res.', 'citations per faculty', 'age'])

Het valt op dat meerdere scholen binnen de tweede dataset afkortingen tussen haakjes achter de naam hebben staan of 'The'. Daarom pas ik deze namen aan zodat ze straks beter te mergen zijn.

In [13]:
df2['title'] = df2['title'].str.replace("\s\(.+\)", "", case=False, regex=True)
df2['title'] = df2['title'].str.replace("The ", "", case=False, regex=True)

In [14]:
df1.head(25)

,title,country,number students,students staff ratio,perc intl students,gender ratio,overall score,teaching score,research score,citations score,industry income score,intl outlook score
0,University of Oxford,United Kingdom,"20,774",11.1,41%,46 : 54,95.6,91.3,99.6,98.0,68.7,96.4
1,Stanford University,United States,"16,223",7.4,23%,44 : 56,94.9,92.2,96.7,99.9,90.1,79.5
2,Harvard University,United States,"21,261",9.3,25%,49 : 51,94.8,94.4,98.8,99.4,46.8,77.7
3,California Institute of Technology,United States,"2,238",6.3,33%,36 : 64,94.5,92.5,96.9,97.0,92.7,83.6
4,Massachusetts Institute of Technology,United States,"11,276",8.4,34%,39 : 61,94.4,90.7,94.4,99.7,90.4,90.0
5,University of Cambridge,United Kingdom,"19,37",11.0,38%,47 : 53,94.0,90.3,99.2,95.6,52.1,95.7
6,"University of California, Berkeley",United States,"39,918",19.8,17%,51 : 49,92.2,85.8,97.2,99.1,84.3,72.3
7,Yale University,United States,"12,91",6.0,20%,50 : 50,91.6,91.9,93.8,97.9,56.1,68.4
8,Princeton University,United States,"8,091",8.0,23%,46 : 54,91.5,88.8,92.5,98.9,58.0,80.2
9,The University of Chicago,United States,"14,292",5.9,31%,46 : 54,90.3,88.9,90.5,98.6,54.9,74.0


In [15]:
df2.head(25)

,title,academic reputation,employer reputation,faculty student,international faculty,international students,overall
0,Massachusetts Institute of Technology,100.0,100.0,100.0,100.0,91.4,100
1,University of Oxford,100.0,100.0,100.0,99.5,98.5,99.5
2,Stanford University,100.0,100.0,100.0,99.8,67.0,98.7
3,University of Cambridge,100.0,100.0,100.0,100.0,97.7,98.7
4,Harvard University,100.0,100.0,99.1,84.2,70.1,98
5,California Institute of Technology,96.7,89.9,100.0,99.4,87.7,97.4
6,Imperial College London,98.4,99.8,99.8,100.0,100.0,97.3
7,ETH Zurich - Swiss Federal Institute of Techno...,98.7,95.3,80.4,100.0,98.2,95.4
8,UCL,99.4,98.9,99.0,99.5,100.0,95.4
9,University of Chicago,99.2,93.5,95.5,71.9,84.9,94.5


Het is lastig te zien of nu de meeste namen kloppen, maar als het goed is is het grootste gedeelte van de top 25 goed gegaan. Daarom besluit ik de twee datasets te mergen met elkaar. 

In [16]:
df = df1.merge(df2, on='title')

In [19]:
df.head(20)

,title,country,number students,students staff ratio,perc intl students,gender ratio,overall score,teaching score,research score,citations score,industry income score,intl outlook score,academic reputation,employer reputation,faculty student,international faculty,international students,overall
0,University of Oxford,United Kingdom,"20,774",11.1,41%,46 : 54,95.6,91.3,99.6,98.0,68.7,96.4,100.0,100.0,100.0,99.5,98.5,99.5
1,Stanford University,United States,"16,223",7.4,23%,44 : 56,94.9,92.2,96.7,99.9,90.1,79.5,100.0,100.0,100.0,99.8,67.0,98.7
2,Harvard University,United States,"21,261",9.3,25%,49 : 51,94.8,94.4,98.8,99.4,46.8,77.7,100.0,100.0,99.1,84.2,70.1,98
3,California Institute of Technology,United States,"2,238",6.3,33%,36 : 64,94.5,92.5,96.9,97.0,92.7,83.6,96.7,89.9,100.0,99.4,87.7,97.4
4,Massachusetts Institute of Technology,United States,"11,276",8.4,34%,39 : 61,94.4,90.7,94.4,99.7,90.4,90.0,100.0,100.0,100.0,100.0,91.4,100
5,University of Cambridge,United Kingdom,"19,37",11.0,38%,47 : 53,94.0,90.3,99.2,95.6,52.1,95.7,100.0,100.0,100.0,100.0,97.7,98.7
6,Yale University,United States,"12,91",6.0,20%,50 : 50,91.6,91.9,93.8,97.9,56.1,68.4,99.9,100.0,100.0,87.9,69.8,90.2
7,Princeton University,United States,"8,091",8.0,23%,46 : 54,91.5,88.8,92.5,98.9,58.0,80.2,99.9,99.2,68.7,28.1,63.2,88.6
8,Imperial College London,United Kingdom,"17,176",11.6,58%,39 : 61,89.4,82.3,88.2,97.2,69.6,97.4,98.4,99.8,99.8,100.0,100.0,97.3
9,Johns Hopkins University,United States,"16,432",4.4,27%,52 : 48,89.2,81.6,91.8,97.7,93.4,73.9,87.6,51.6,100.0,55.1,91.7,85.9


## 3. Feature engineering

explain which transformations you have made to make your variables more informative (e.g., calculating number of days from a starting date).

UITTYPEN
Twee datasets samengevoegd om te kijken naar de verschillende informatie

!!! Beide organisaties berekenen namelijk de overall score met percentages. Zo telt het onderzoeksgedeelte meer mee dan het aantal internationale studenten. Daarom bereken ik mijn eigen overall score.

In [18]:
df['gemiddelde'] = df['overall score', 'overall'].mean()

KeyError: ('overall score', 'overall')

## 4. Exploratory data analysis

explore your data set with the research question in mind. Present relevant graphs, tables and numbers with respect to your problem.

In [ ]:
df2.head()

In [ ]:
df1.info()

In [ ]:
df = df1.merge(df2, on='title')

In [ ]:
df.info()

In [ ]:
df.head(600)

Het valt al bij de eerste 10 rijen op dat er verschillende rijen zijn weggevallen, bijvoorbeeld de Massachusetts Institute of Technology (MIT) van de tweede dataset. Daarom pak ik de gemiddelde scoren van de overall_score van df1 en df 2 en sorteer ik vanaf daar op score.

In [ ]:
df['overall score'].astype(float)
df['mean'] = df[['overall score', 'overall']].mean(numeric_only=False)

In [ ]:
df.head()